# Explain an Intermediate Layer of VGG16 on ImageNet (PyTorch)

Explaining a prediction in terms of the original input image is harder than explaining the predicition in terms of a higher convolutional layer (because the higher convolutional layer is closer to the output). This notebook gives a simple example of how to use GradientExplainer to do explain a model output with respect to the 7th layer of the pretrained VGG16 network.

Note that by default 200 samples are taken to compute the expectation. To run faster you can lower the number of samples per explanation.

In [ ]:
import torch, torchvision
from torch import nn
from torchvision import transforms, models, datasets
import shap
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def normalize(image):
    if image.max() > 1:
        image /= 255
    image = (image - mean) / std
    # in addition, roll the axis so that they suit pytorch
    return torch.tensor(image.swapaxes(-1, 1).swapaxes(2, 3)).float()

In [ ]:
from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey

model, transformer, device = init_from_onekey(r'D:/20230705-ZhaoFaFa/label1/models2d/resnet101/viz')

In [ ]:
import os
import numpy as np
from PIL import Image

root = r'D:/20230705-ZhaoFaFa/Data_Z_HE/crop/'
# load the model
model = model.eval()
# model = models.vgg16(pretrained=True).eval()
save_dir = r'D:/20230705-ZhaoFaFa/viz/'
os.makedirs(save_dir, exist_ok='True')
for sample in os.listdir(root):
    if '+02' in sample or '-02' in sample:
        continue
    print(f"正在预测：{sample}")
    samples = [np.expand_dims(np.array(Image.open(os.path.join(root, sample)).convert('RGB').resize((224, 224))), axis=0)]
    samples = np.concatenate(samples, axis=0)
    X = samples / 255
    to_explain = X

    e = shap.GradientExplainer((model, model.conv1), normalize(X))
    shap_values, indexes = e.shap_values(normalize(to_explain), ranked_outputs=2, nsamples=50)

    # get the names for the classes
    index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

    # plot the explanations
    shap_values = [np.swapaxes(np.swapaxes(s, 2, 3), 1, -1) for s in shap_values]

    shap.image_plot(shap_values, to_explain, np.array([['none Increase', 'Increase']]), show=False)
    plt.savefig(f'{save_dir}/{os.path.splitext(sample)[0]}_shap_viz.svg')

    plt.close()

In [ ]:
model.conv1

In [ ]:
model = models.vgg16(pretrained=True).eval()
model.features[7]

## Explain with local smoothing

Gradient explainer uses expected gradients, which merges ideas from integrated gradients, SHAP, and SmoothGrad into a single expection equation. To use smoothing like SmoothGrad just set the local_smoothing parameter to something non-zero. This will add normally distributed noise with that standard deviation to the input during the expectation calculation. It can create smoother feature attributions that better capture correlated regions of the image.

In [ ]:
# note that because the inputs are scaled to be between 0 and 1, the local smoothing also has to be
# scaled compared to the Keras model
explainer = shap.GradientExplainer((model, model.conv1), normalize(X), local_smoothing=0.5)
shap_values,indexes = explainer.shap_values(normalize(to_explain), ranked_outputs=2, nsamples=200)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

# plot the explanations
shap_values = [np.swapaxes(np.swapaxes(s, 2, 3), 1, -1) for s in shap_values]

shap.image_plot(shap_values, to_explain, np.array([['none Increase', 'Increase'], ['none Increase', 'Increase']]), plot=False)
plt.savefig('img/shap_smoothing_viz.svg')
plt.show()